In [ ]:
# YOLOv8 Training Script

from ultralytics import YOLO
import os
from datetime import datetime

# Load a pretrained YOLOv8n variant (use either a base model or a fine-tuned checkpoint)
# model = YOLO('../src/human_detector/models/yolov11n.pt')  # start from scratch
model = YOLO('/home/yogee/Desktop/human_detector_ws/data_training/results/yolov11n_ep50_imgsz416_bs2_20250523_1040/weights/best_week2.pt')  # resume from previous best

# Training configuration
epochs = 50         # Number of full training passes over the dataset
imgsz = 640          # Input image resolution (higher gives better detail at cost of speed/memory)
batch_size = 4       # Number of images processed together in one step (depends on GPU memory)

# Dynamically name experiment folder based on parameters and timestamp
time_stamp = datetime.now().strftime("%Y%m%d_%H%M")
exp_name = f"yolov11n_ep{epochs}_imgsz{imgsz}_bs{batch_size}_{time_stamp}"

# Train the model
results = model.train(
    data='data.yaml',          # Path to dataset config file (defines classes and image locations)
    epochs=epochs,             # Total training epochs
    patience=10,               # Early stopping patience (if no improvement in val loss)
    dropout=0.1,               # Dropout rate to prevent overfitting
    imgsz=imgsz,               # Image size for training
    batch=batch_size,          # Batch size
    device=0,                  # Device index (0 = first GPU)
    half=True,                 # Use float16 precision for faster training (if supported by GPU)
    workers=2,                 # Number of data loading workers (increase if CPU allows)
    cache=True,                # Cache images in memory for faster access during training

    # Data augmentation settings (improve generalization and robustness)
    hsv_h=0.015,               # Adjust hue slightly
    hsv_s=0.7,                 # Saturation change (color intensity)
    hsv_v=0.4,                 # Brightness (value) change
    translate=0.2,             # Random image shift (as % of image size)
    scale=0.4,                 # Random image scaling (zoom in/out)
    fliplr=0.5,                # Probability of horizontal flip
    mosaic=0.8,                # Use mosaic augmentation (combines 4 images into one)
    mixup=0.2,                 # Mix two images together to help prevent overfitting
    copy_paste=0.2,            # Overlay one object onto another image to simulate occlusion
    perspective=0.01,         # Slight image perspective warping for robustness

    # Output settings
    project='results',         # Folder to save training results
    name=exp_name              # Subfolder named with experiment configuration
)

# Save path of the best trained model
save_dir = str(results.save_dir)
best_pt = os.path.join(save_dir, 'weights', 'best.pt')
print(f"Trained checkpoint saved to: {best_pt}")


New https://pypi.org/project/ultralytics/8.3.143 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.141 🚀 Python-3.10.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce MX550, 1864MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=/home/yogee/Desktop/human_detector_ws/data_training/results/yolov11n_ep50_imgsz416_bs2_20250523_1040/weights/best.pt, momentum=0.

train: Scanning /home/yogee/Desktop/human_detector_ws/data_training/data_person/train/labels... 2793 images, 23 backgrounds, 0 corrupt: 100%|██████████| 2793/2793 [00:02<00:00, 1290.69it/s]

train: New cache created: /home/yogee/Desktop/human_detector_ws/data_training/data_person/train/labels.cache


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.2GB RAM): 100%|██████████| 2793/2793 [00:06<00:00, 453.94it/s] 


...

      30/30     0.811G        nan        nan        nan          2        640: 100%|██████████| 699/699 [03:16<00:00,  3.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 176/176 [00:23<00:00,  7.39it/s]

                   all       1407       3358          0          0          0          0



30 epochs completed in 1.873 hours.
Optimizer stripped from results/yolov11n_ep30_imgsz640_bs4_20250523_14022/weights/last.pt, 5.4MB
Optimizer stripped from results/yolov11n_ep30_imgsz640_bs4_20250523_14022/weights/best.pt, 5.4MB

Validating results/yolov11n_ep30_imgsz640_bs4_20250523_14022/weights/best.pt...
Ultralytics 8.3.141 🚀 Python-3.10.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce MX550, 1864MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 176/176 [00:22<00:00,  7.98it/s]


                   all       1407       3358          0          0          0          0
                person       1250       2734          0          0          0          0
                teleco        331        624          0          0          0          0
Speed: 0.3ms preprocess, 14.6ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to results/yolov11n_ep30_imgsz640_bs4_20250523_14022
Trained checkpoint saved to: results/yolov11n_ep30_imgsz640_bs4_20250523_14022/weights/best.pt


# 🧠 YOLOv8 Training Metrics Cheatsheet

## 🔢 Loss Curves

| Metric            | Meaning                                                | Goal / Good Trend                |
|-------------------|--------------------------------------------------------|----------------------------------|
| `train/box_loss`  | Bounding box regression loss (training)                | ↓ Steady decline                 |
| `train/cls_loss`  | Classification loss (training)                         | ↓ Should fall and stabilize     |
| `train/dfl_loss`  | Distribution Focal Loss (for bounding box quality)     | ↓ Smooth drop                   |
| `val/box_loss`    | Box loss on validation set                             | ↓ Best indicator of generalization |
| `val/cls_loss`    | Classification loss on validation                      | ↓ Falling → Good model fit      |
| `val/dfl_loss`    | DFL loss on validation                                 | ↓ Matches train trend = no overfit |

---

## 📈 Detection Metrics

| Metric                | Description                                                 | Goal / Interpretation           |
|------------------------|-------------------------------------------------------------|----------------------------------|
| `metrics/precision(B)` | % of correct detections out of all predictions              | ↑ High = few false positives     |
| `metrics/recall(B)`    | % of actual objects detected                                | ↑ High = few missed detections   |
| `metrics/mAP50(B)`     | Accuracy of boxes at 50% IoU threshold                     | ↑ Main object detection metric   |
| `metrics/mAP50-95(B)`  | Average mAP over IoU 0.5 to 0.95 (stricter)                | ↑ Robust performance measure     |

---

## 🧪 Ideal Curve Patterns

- **Losses**: should steadily decrease.
- **Validation loss** rising while training loss decreases → possible **overfitting**.
- **Precision & Recall**: should increase and plateau near 1.0.
- **mAP**: aim for **>0.90 mAP50** and **>0.80 mAP50-95** for high-quality results.

---

## ⚠️ Tips

- 🧊 Use `half=True` for mixed precision training on supported GPUs.
- 🧠 Watch `val/box_loss` and `mAP50-95` to catch overfitting early.
- 🧪 If metrics plateau early, try increasing `epochs`, `imgsz`, or improving labels.
- 💾 Check `runs/train/.../results.png` for curve visualization.

---
